
Preconditions:

1. Write image to SD card - compeleted

2. New primary partition of ext4 format


```shell

# 1. check /dev/mmcblk0p3 is existed

  fdisk -l /dev/mmcblk0 | grep /dev/mmcblk0p3

# 2. re-format the new primary partition (just for realiablity)

  mkfs -t ext4 /dev/mmcblk0p3

# 3. mount this new primary partition

  mkdir /var_temp
  
  mount /dev/mmcblk0p3 /var_temp

# 4. backup /var /home /srv and copy /var to the new primary partition 

  mkdir /var_bkup /home_bkup /srv_bkup
  
  cp -r -p /var/* /var_bkup/
  
  cp -r -p /home/* /home_bkup/
  
  #cp -r -p /srv/* /srv_bkup/

  cp -r -p /var/* /var_temp/

# 5. make necessary directories and copy necessary files into them
  
  mkdir -p /var/local/home /var/local/srv /var/local/hyt

  mkdir -p /var_temp/local/home /var_temp/local/srv /var_temp/local/hyt

  cp -r -p /home/* /var_temp/local/home
  
  cp -r -p /srv/* /var_temp/local/srv

# 6. edit /etc/fstab

cat << "EOF" > /etc/fstab
proc            /proc           proc    defaults          0       0
/dev/mmcblk0p1  /boot           vfat    defaults          0       2
/dev/mmcblk0p2  /               ext4    defaults,noatime  0       1
/dev/mmcblk0p3  /var            ext4    defaults,noatime  0       0
/var/local/home /home           none    defaults,bind     0       0
/var/local/srv  /srv            none    defaults,bind     0       0
/var/local/hyt  /hyt            none    defaults,bind     0       0
# a swapfile is not a swap partition, so no using swapon|off from here on, use  dphys-swapfile swap[on|off]  for that
EOF

# 7. change package sources

cat << EOF > /etc/apt/sources.list
# alicloud
deb http://mirrors.aliyun.com/raspbian/raspbian/ jessie main non-free contrib rpi
deb-src http://mirrors.aliyun.com/raspbian/raspbian/ jessie main non-free contrib rpi

# original
#deb http://mirrordirector.raspbian.org/raspbian/ wheezy main contrib non-free rpi
EOF

# 8. update packages

  apt-get remove --purge \ 
    wolfram-engine \
    triggerhappy \
    lightdm
    
  apt-get clean
  
  apt-get autoremove
  
# 9. turn off using swap

  dphys-swapfile swapoff
  dphys-swapfile uninstall
  update-rc.d dphys-swapfile remove

  # just have a check
  free -m

```
